<h3>Parte 1</h3><br>
<p>Fazendo scrapy dos dados e salvando em json.</p>

In [2]:
import os
import subprocess

def execute_commands(comando):
    retorno = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    if retorno.returncode == 0:
        print(f"Comando '{comando}' executado com sucesso!")
    else:
        print(f"Erro na execução de: {comando}")
        print(retorno.stderr.decode())

# acessar diretorio do projeto
os.chdir("..\scrapcards")

# comandos para executar respectivos spiders
comandos = ["scrapy crawl ligamagic -O resultmtg.json", "scrapy crawl ligapoke -O resultpoke.json", "scrapy crawl ligaygo -O resultygo.json"]

for comando in comandos:
    execute_commands(comando)   


Comando 'scrapy crawl ligamagic -O resultmtg.json' executado com sucesso!
Comando 'scrapy crawl ligapoke -O resultpoke.json' executado com sucesso!
Comando 'scrapy crawl ligaygo -O resultygo.json' executado com sucesso!


<h3>Parte 2</h3><br>
<p>Preparando os dados</p>